In [5]:
%matplotlib inline
#matplotlib.use('Agg')
import d2l
from mxnet import autograd,nd,gluon
from IPython import display

batch_size=256
train_iter,test_iter=d2l.load_data_fashion_mnist(batch_size)

num_inputs=784
num_outputs=10

W=nd.random.normal(scale=0.01,shape=(num_inputs,num_outputs))
b=nd.zeros(num_outputs)

W.attach_grad()
b.attach_grad()


def softmax(X):
    X_exp=X.exp()
    partition=X_exp.sum(axis=1,keepdims=True)
    return X_exp/partition

def net(X):
    return softmax(nd.dot(X.reshape((-1,num_inputs)),W)+b) #-1 means unspecified num. +b:propagation

def cross_entropy(y_hat,y):
    return -nd.pick(y_hat,y).log()

def accuracy(y_hat, y):
    return (y_hat.argmax(axis=1) == y.astype('float32')).sum().asscalar()

def evaluate_accuracy(net,data_iter):
    metric=d2l.Accumulator(2) # num_corrected_examples, num_examples
    for X,y in data_iter:
        #print(X.shape)  shape is (256, 1, 28, 28)
        #print(y)  e.g. [1,0,6,9....7]
        y=y.astype('float32')
        metric.add(accuracy(net(X),y),y.size) #y.size = size of this batch, e.g.256
    return metric[0]/metric[1]

class Accumulator(object):
    """Sum a list of numbers over time"""
    def __init__(self, n):
        self.data = [0.0] * n
    def add(self, *args):
        self.data = [a+b for a, b in zip(self.data, args)] #args is array, so it's array+array
    def reset(self):
        self.data = [0] * len(self.data)
    def __getitem__(self, i):
        return self.data[i]


print(evaluate_accuracy(net, test_iter)) 

# Save to the d2l package.
def train_epoch_ch3(net, train_iter, loss, updater):
    metric = Accumulator(3) # train_loss_sum, train_acc_sum, num_examples
    if isinstance(updater, gluon.Trainer):
        updater = updater.step
    for X, y in train_iter:
        # compute gradients and update parameters
        with autograd.record():
            y_hat = net(X)
            l = loss(y_hat, y)
        l.backward()
        updater(X.shape[0])
        metric.add(l.sum().asscalar(), accuracy(y_hat, y), y.size)
    # Return training loss and training accuracy
    return metric[0]/metric[2], metric[1]/metric[2]


# Save to the d2l package.
class Animator(object):
    def __init__(self, xlabel=None, ylabel=None, legend=[], xlim=None,
                 ylim=None, xscale='linear', yscale='linear', fmts=None,
                 nrows=1, ncols=1, figsize=(3.5, 2.5)):
        """Incrementally plot multiple lines."""
        d2l.use_svg_display()
        self.fig, self.axes = d2l.plt.subplots(nrows, ncols, figsize=figsize)
        if nrows * ncols == 1: self.axes = [self.axes,]
        # use a lambda to capture arguments
        self.config_axes = lambda : d2l.set_axes(
            self.axes[0], xlabel, ylabel, xlim, ylim, xscale, yscale, legend)
        self.X, self.Y, self.fmts = None, None, fmts

    def add(self, x, y):
        """Add multiple data points into the figure."""
        if not hasattr(y, "__len__"): y = [y]
        n = len(y)
        if not hasattr(x, "__len__"): x = [x] * n
        if not self.X: self.X = [[] for _ in range(n)]
        if not self.Y: self.Y = [[] for _ in range(n)]
        if not self.fmts: self.fmts = ['-'] * n
        for i, (a, b) in enumerate(zip(x, y)):
            if a is not None and b is not None:
                self.X[i].append(a)
                self.Y[i].append(b)
        self.axes[0].cla()
        for x, y, fmt in zip(self.X, self.Y, self.fmts):
            self.axes[0].plot(x, y, fmt)
        self.config_axes()
        display.display(self.fig)
        display.clear_output(wait=True)


# Save to the d2l package.
def train_ch3(net, train_iter, test_iter, loss, num_epochs, updater):
    trains, test_accs = [], []
    animator = Animator(xlabel='epoch', xlim=[1, num_epochs],
                        ylim=[0.3, 0.9],
                        legend=['train loss', 'train acc', 'test acc'])
    for epoch in range(num_epochs):
        train_metrics = train_epoch_ch3(net, train_iter, loss, updater)
        test_acc = evaluate_accuracy(net, test_iter)
        animator.add(epoch+1, train_metrics+(test_acc,))
       
        
num_epochs, lr = 10, 0.1
updater = lambda batch_size: d2l.sgd([W, b], lr, batch_size)
#train_ch3(net, train_iter, test_iter, cross_entropy, num_epochs, updater)


# Save to the d2l package.
def predict_ch3(net, test_iter, n=6):
    for X, y in test_iter:
        break
    trues = d2l.get_fashion_mnist_labels(y.asnumpy())
    preds = d2l.get_fashion_mnist_labels(net(X).argmax(axis=1).asnumpy())
    titles = [true+'\n'+ pred for true, pred in zip(trues, preds)]
    d2l.show_images(X[0:n].reshape((n,28,28)), 1, n, titles=titles[0:n])

    
    
    
#predict_ch3(net, test_iter)

(256, 1, 28, 28)
(256, 1, 28, 28)
(256, 1, 28, 28)
(256, 1, 28, 28)
(256, 1, 28, 28)
(256, 1, 28, 28)
(256, 1, 28, 28)
(256, 1, 28, 28)
(256, 1, 28, 28)
(256, 1, 28, 28)
(256, 1, 28, 28)
(256, 1, 28, 28)
(256, 1, 28, 28)
(256, 1, 28, 28)
(256, 1, 28, 28)
(256, 1, 28, 28)
(256, 1, 28, 28)
(256, 1, 28, 28)
(256, 1, 28, 28)
(256, 1, 28, 28)
(256, 1, 28, 28)
(256, 1, 28, 28)
(256, 1, 28, 28)
(256, 1, 28, 28)
(256, 1, 28, 28)
(256, 1, 28, 28)
(256, 1, 28, 28)
(256, 1, 28, 28)
(256, 1, 28, 28)
(256, 1, 28, 28)
(256, 1, 28, 28)
(256, 1, 28, 28)
(256, 1, 28, 28)
(256, 1, 28, 28)
(256, 1, 28, 28)
(256, 1, 28, 28)
(256, 1, 28, 28)
(256, 1, 28, 28)
(256, 1, 28, 28)
(16, 1, 28, 28)
0.0842
